In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
ens_2016 = pd.read_excel('DataBase/ENS_chile_2016-2017_expansion_factors.xlsx')
ens_2016 = ens_2016.iloc[:,1::]
ens_2016;

In [ ]:
ens_2016['chronic_pain'] = np.where(ens_2016['pain_time_overall'] >= 90, 1, 0)

In [ ]:
ens_2016 = ens_2016[ens_2016['age'] <= 80]

In [ ]:
ens_2016 = ens_2016[ens_2016['has_cancer'] == 2]

In [ ]:
ens_2016['reduc_mob'] = ens_2016['reduc_mob'].apply(lambda x: 1 if x == 1 else 0)
ens_2016['depression'] = ens_2016['depression'].apply(lambda x: 1 if x == 1 else 0)
ens_2016['anxiety'] = ens_2016['anxiety'].apply(lambda x: 1 if x == 1 else 0)
ens_2016['malnutrition'] = ens_2016['malnutrition'].apply(lambda x: 1 if x == 1 else 0)

In [ ]:
df = ens_2016[['chronic_pain', 'sex', 'age', 'reduc_mob', 'depression', 'anxiety', 'malnutrition', 'educ_yrears', 'SES', 'factor']]
df = df.copy()

In [ ]:
df.dropna(inplace=True)

In [ ]:
Counter(df['chronic_pain'])

In [ ]:
from __future__ import division
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
from xgboost import plot_importance
import plotly.graph_objects as go
from tqdm.notebook import tqdm
from sklearn.model_selection import RepeatedKFold
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score, precision_score, confusion_matrix, mean_squared_error
import seaborn as sns
from statannot import add_stat_annotation
from sklearn.linear_model import Lasso, MultiTaskLasso, Ridge, ElasticNet

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression, f_regression

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.decomposition import PCA

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

from sklearn import metrics


from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import statsmodels.api as sm

import time

import scipy

from sklearn.svm import SVR

from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestRegressor

from statsmodels.stats.multitest import fdrcorrection

from statsmodels.stats import multitest

from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso
from sklearn.ensemble import ExtraTreesClassifier

from collections import Counter
import heapq

import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor

import re

from sklearn.preprocessing import MinMaxScaler

import os

In [ ]:
def classifier_ROC(X_, y, fig_show = True, params_ = None, rand_s = 10):
    
    
    
    X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.1, random_state=rand_s, stratify=y)
    
    
    X_train_weights =  X_train['factor']
    X_test_weights =  X_test['factor']
    
    
    #X_train = X_train[['sex', 'age', 'educ_yrears', 'total_local_pain_without_lat', 'SES']]
    #X_test  = X_test[['sex', 'age', 'educ_yrears', 'total_local_pain_without_lat', 'SES']]
    


    #lista = list(sfs_.k_feature_names_)

    #X_train = X_train[lista]
    #X_test = X_test[lista]

    cv    = RepeatedKFold(n_splits=3, n_repeats=2, random_state=101)
    folds = [(train,test) for train, test in cv.split(X_train, y_train)]
    metrics = ['auc', 'fpr', 'tpr', 'thresholds', 'f1_score', 'accuracy_score', 'recall_score', 'precision_score', 'confusion_matrix', 'model']
    results = {
        'train': {m:[] for m in metrics},
        'val'  : {m:[] for m in metrics},
        'test' : {m:[] for m in metrics}
    }
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train)


    if(params_!=None):
        params = params_
    else:
        params = {
            'objective'   : 'binary:logistic',
            'eval_metric' : 'logloss'
        }


    y_preds_ = []
    labels_ = []
    
    plt.rcParams["figure.figsize"] = (10,10)
    dtest = xgb.DMatrix(X_test, label=y_test, weight=X_test_weights)
    for train, test in tqdm(folds, total=len(folds)):
        
        #print(X_train_weights, type(X_train_weights), X_train_weights.iloc)
        
        
        
        dtrain = xgb.DMatrix(X_train.iloc[train,:], label=y_train.iloc[train], weight=X_train_weights.iloc[train])
        dval   = xgb.DMatrix(X_train.iloc[test,:], label=y_train.iloc[test], weight=X_train_weights.iloc[test])
        model  = xgb.train(
            dtrain                = dtrain,
            params                = params, 
            evals                 = [(dtrain, 'train'), (dval, 'val')],
            num_boost_round       = 1000,
            verbose_eval          = False,
            early_stopping_rounds = 10,
        )
        sets = [dtrain, dval, dtest]

        for i,ds in enumerate(results.keys()):
            y_preds              = model.predict(sets[i])
            labels               = sets[i].get_label()
            fpr, tpr, thresholds = roc_curve(labels, y_preds)
            results[ds]['fpr'].append(fpr)
            results[ds]['tpr'].append(tpr)
            results[ds]['thresholds'].append(thresholds)
            results[ds]['auc'].append(roc_auc_score(labels, y_preds))

            results[ds]['f1_score'].append(f1_score(labels, np.round(y_preds)))
            results[ds]['accuracy_score'].append(accuracy_score(labels, np.round(y_preds)))
            results[ds]['recall_score'].append(recall_score(labels, np.round(y_preds)))
            results[ds]['precision_score'].append(precision_score(labels, np.round(y_preds)))
            results[ds]['confusion_matrix'].append(confusion_matrix(labels, np.round(y_preds)))
            results[ds]['model'].append(model)
            
            
            if(ds == 'test'):
                y_preds_.extend(y_preds)
                labels_.extend(labels)
                
   # display(y_preds_)
   # display(labels_)
    print(roc_auc_score(labels_, y_preds_), rand_s)
    
    kind = 'test'
    c_fill      = 'rgba(52, 152, 219, 0.2)'
    c_line      = 'rgba(52, 152, 219, 0.5)'
    c_line_main = 'rgba(41, 128, 185, 1.0)'
    c_grid      = 'rgba(189, 195, 199, 0.5)'
    c_annot     = 'rgba(149, 165, 166, 0.5)'
    c_highlight = 'rgba(192, 57, 43, 1.0)'

    fpr_mean    = np.linspace(0, 1, 100)
    interp_tprs = []

    for i in range(3*2):
        fpr           = results[kind]['fpr'][i]
        tpr           = results[kind]['tpr'][i]
        interp_tpr    = np.interp(fpr_mean, fpr, tpr)
        interp_tpr[0] = 0.0
        interp_tprs.append(interp_tpr)

    tpr_mean     = np.mean(interp_tprs, axis=0)
    tpr_mean[-1] = 1.0
    tpr_std      = 2*np.std(interp_tprs, axis=0)
    tpr_upper    = np.clip(tpr_mean+tpr_std, 0, 1)
    tpr_lower    = tpr_mean-tpr_std
    auc          = np.mean(results[kind]['auc'])
    auc_std          = np.std(results[kind]['auc'])
    
    roc_dict = {}
    roc_dict['tpr_mean'] = tpr_mean
    roc_dict['tpr_std'] = tpr_std
    roc_dict['tpr_upper'] = tpr_upper
    roc_dict['tpr_lower'] = tpr_lower
    roc_dict['auc'] = auc
    roc_dict['auc_std'] = auc_std

    if(fig_show):
        fig = go.Figure([
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_upper,
                line       = dict(color=c_line, width=1),
                hoverinfo  = "skip",
                showlegend = False,
                name       = 'upper'),
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_lower,
                fill       = 'tonexty',
                fillcolor  = c_fill,
                line       = dict(color=c_line, width=1),
                hoverinfo  = "skip",
                showlegend = False,
                name       = 'lower'),
            go.Scatter(
                x          = fpr_mean,
                y          = tpr_mean,
                line       = dict(color=c_line_main, width=2),
                hoverinfo  = "skip",
                showlegend = True,
                name       = f'AUC: {auc:.3f}')
        ])

        fig.add_shape(
            type ='line', 
            line =dict(dash='dash'),
            x0=0, x1=1, y0=0, y1=1
        )

        fig.update_layout(
            template    = 'plotly_white', 
            title_x     = 0.5,
            xaxis_title = "1 - Specificity",
            yaxis_title = "Sensitivity",
            width       = 800,
            height      = 800,
            legend      = dict(
                yanchor="bottom", 
                xanchor="right", 
                x=0.95,
                y=0.01,
            )
        )

        fig.update_yaxes(
            range       = [0, 1],
            gridcolor   = c_grid,
            scaleanchor = "x", 
            scaleratio  = 1,
            linecolor   = 'black')

        fig.update_xaxes(
            range       = [0, 1],
            gridcolor   = c_grid,
            constrain   = 'domain',
            linecolor   = 'black')



        fig.show()
        
    else:
        print(y.name, 'AUC: ', np.round(auc,2), '+-( ', np.round(np.std(results[kind]['auc']),2), ')', X_.shape)
        print('acc={:.2f} prec={:.2f}  f1={:.2f} rec={:.2f}'.format(np.mean(results[ds]['accuracy_score']), np.mean(results[ds]['precision_score']),np.mean(results[ds]['f1_score']), np.mean(results[ds]['recall_score'])))

    return (model, results, np.round(auc,2), roc_dict)

In [ ]:
from skopt import BayesSearchCV
def SearchBestModel(case_x, case_y):

    X_train, X_test, y_train, y_test = train_test_split(case_x, case_y, test_size=.1, random_state=1)

    opt_XGB = BayesSearchCV(
            XGBClassifier(),
            {
                'objective'   : ['binary:logistic'],
                'eval_metric' : ['logloss', 'error', 'auc', 'aucpr'],
                'learning_rate': (0.01, 0.1, 0.2),
                'max_depth': (3, 4, 5),
                'subsample': (0.8, 0.9, 1.0),
                'colsample_bytree': (0.8, 0.9, 1.0),
                'reg_alpha': (0, 0.1, 1.0),
                'reg_lambda': (0, 0.1, 1.0),
                #'gamma': (0.001, 0.01, 0.1, 1, 10),
            },
                          
            #n_iter=10,
            #cv=3,
            random_state=1
        )

    opt_XGB.fit(X_train, y_train)


    print('XGB')
    print("val. score: %s" % opt_XGB.best_score_)
    print("test score: %s" % opt_XGB.score(X_test, y_test))
    print("best parameters: %s" % str(opt_XGB.best_params_))
    print('---------------------------------------------\n')
    
    return opt_XGB

In [ ]:
def graph_feat_import(gini_results_metric, top_10_keys_metric, colores = ['#f25c05', '#9c0720'], gini_plus_var = True, title = ''):

    count = 1
    for i in top_10_keys_metric:

        #auc = gini_results_metric[i][3]['auc']
#
        #acc = np.mean(gini_results_metric[i][1]['test']['accuracy_score'])
        #prec = np.mean(gini_results_metric[i][1]['test']['precision_score'])
        #f1 = np.mean(gini_results_metric[i][1]['test']['f1_score'])
        
        
        auc = np.mean(gini_results_metric[i][1]['test']['auc'])

        acc = np.mean(gini_results_metric[i][1]['test']['accuracy_score'])
        prec = np.mean(gini_results_metric[i][1]['test']['precision_score'])
        f1 = np.mean(gini_results_metric[i][1]['test']['f1_score'])
        rec = np.mean(gini_results_metric[i][1]['test']['recall_score'])

        auc_std = np.std(gini_results_metric[i][1]['test']['auc'])
        acc_std = np.std(gini_results_metric[i][1]['test']['accuracy_score'])
        prec_std = np.std(gini_results_metric[i][1]['test']['precision_score'])
        f1_std = np.std(gini_results_metric[i][1]['test']['f1_score'])
        rec_std = np.std(gini_results_metric[i][1]['test']['recall_score'])
        
        conf_matrix = gini_results_metric[i][1]['test']['confusion_matrix']

        conf_matrix_mean = np.mean(conf_matrix, axis=0)
        conf_matrix_std  = np.std(conf_matrix, axis=0)

        fscore = {}
        for k in ['sex', 'age', 'reduc_mob', 'depression', 'anxiety', 'malnutrition', 'educ_yrears', 'SES']:
            fscore[k] = []

        for model in gini_results_metric[i][1]['test']['model']:

            for j in ['sex', 'age', 'reduc_mob', 'depression', 'anxiety', 'malnutrition', 'educ_yrears', 'SES']:
                try:
                    fscore[j].append(model.get_fscore()[j])
                except:
                    pass
                

        if(count<=4):
            plt.subplot(2,1,count)   
            max_length = max(len(v) for v in fscore.values())

            fscore_ = {k: v + [None] * (max_length - len(v)) for k, v in fscore.items()}

            summary_df = pd.DataFrame(fscore_)

            summary_df = pd.DataFrame({
                'Mean': summary_df.mean(),
                'Std': summary_df.std()
            })


            summary_df = summary_df.sort_values(by='Mean', ascending=True)
            
            
            mean_values = summary_df.Mean
            std_values = summary_df.Std
            
            #display(mean_values)
            #display(std_values)
            
            ax = mean_values.plot(kind='barh', xerr=std_values, capsize=4, color=colores)

            plt.ylabel('Features')
            plt.xlabel('f-score')
            plt.title(''+i + '\n' + ' auc={:.2f}\n acc={:.2f} prec={:.2f} \n f1={:.2f} rec={:.2f}'.format(auc, acc,prec, f1, rec))

            xlimits = ax.get_xlim()
            ax.set_xticks([xlimits[0], xlimits[1]])
            ax.tick_params(axis='x', labelsize=14)
            
            n_row, n_col = conf_matrix_mean.shape
            labels = [[f"{conf_matrix_mean[i, j]:.2f} \n ({conf_matrix_std[i, j]:.2f})" for j in range(n_col)] for i in range(n_row)]


            clases = ['No chronic pain', 'Chronic pain']

            plt.subplot(2,1,count+1) 
            plt.imshow(conf_matrix_mean, interpolation='nearest', cmap=plt.cm.Greys)
            plt.title(''+i)
            plt.colorbar()

            tick_marks = np.arange(len(clases))
            plt.xticks(tick_marks, clases)
            plt.yticks(tick_marks, clases)

            plt.ylabel('True')
            plt.xlabel('Predict')

            for row_ in range(n_row):
                for col_ in range(n_col):
                    plt.text(col_, row_, labels[row_][col_], verticalalignment="center", 
                             horizontalalignment="center", color="white" if conf_matrix_mean[row_, col_] > 30 else "black")


        count+=1
        
        metrics = {
            'AUC': [auc, auc_std],
            'Accuracy': [acc, acc_std],
            'Precision': [prec, prec_std],
            'F1 Score': [f1, f1_std],
            'Recall': [rec, rec_std]
        }
        df_metrics = pd.DataFrame(metrics, index=['Mean', 'STD'])



        clases = ['No chronic pain', 'Chronic pain']


        df_conf_m = pd.DataFrame(conf_matrix_mean, index=clases, columns=clases)
        df_conf_s = pd.DataFrame(conf_matrix_std, index=clases, columns=clases)

In [ ]:
def graph_roc(gini_results_metric, top_10_keys_metric, title = ''):
    #metric_tpr_mean = np.zeros_like(gini_results_metric[i][3]['tpr_mean'])

    metric_tpr_mean = 0
    metric_auc_roc_list = []
    count = 0
    last_i = ''

    acc_list = []
    prec_list = []
    f1_list=[]
    rec_list = []
    for i in top_10_keys_metric:


        tpr = gini_results_metric[i][3]['tpr_mean']
        fpr = np.linspace(0, 1, 100)
        auc_roc = gini_results_metric[i][3]['auc']
        auc_roc_std = gini_results_metric[i][3]['auc_std']

        if(auc_roc<0.50): # Ya los que son menos de 0.75 no los tengo en cuenta así que pinto de naranja el anterior y no guardo más valores para no afectar los promedios
            tpr = gini_results_metric[last_i][3]['tpr_mean']
            fpr = np.linspace(0, 1, 100)
            auc_roc = gini_results_metric[last_i][3]['auc']
            plt.plot(fpr, tpr, color='#FA7F08', alpha=1, lw=1, label='(AUC = {:.2f}) ({:.3f})'.format(auc_roc, auc_roc_std))
            
            l = len(metric_auc_roc_list)
            plt.plot(fpr, metric_tpr_mean/l, color='#0D0D0C', alpha=1, lw=2, label='(AUC = {:.2f} [{:.3f}])'.format(np.mean(metric_auc_roc_list), np.std(metric_auc_roc_list)))
            plt.legend(loc="lower right")

            title_metrics = '\n' + 'acc'
            plt.title('ROC ( '+title+')\n acc={:.2f}  prec={:.2f}  f1={:.2f}'.format(np.mean(acc_list), 
                                                                                        np.mean(prec_list),
                                                                                        np.mean(f1_list)))
            return

        if count==0:
            plt.plot(fpr, tpr, color='#F24405', alpha=1, lw=1, label='(AUC = {:.2f} [{:.3f}]) '.format(auc_roc, auc_roc_std))
        elif count==(len(top_10_keys_metric)-1): 
            plt.plot(fpr, tpr, color='#FA7F08', alpha=1, lw=1, label='(AUC = {:.2f} [{:.3f}])'.format(auc_roc, auc_roc_std))
        else:
            plt.plot(fpr, tpr, color='#747E7E', alpha=0.55, lw=0.5)

        plt.plot([0, 1], [0, 1], 'k--', lw=1.5)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.0])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')


        metric_tpr_mean+=tpr
        metric_auc_roc_list.append(auc_roc)

        acc_list.append(np.mean(gini_results_metric[i][1]['test']['accuracy_score']))
        prec_list.append(np.mean(gini_results_metric[i][1]['test']['precision_score']))
        f1_list.append(np.mean(gini_results_metric[i][1]['test']['f1_score']))
        rec_list.append(np.mean(gini_results_metric[i][1]['test']['recall_score']))
        count+=1
        last_i = i

   #l = len(metric_auc_roc_list)
   #plt.plot(fpr, metric_tpr_mean/l, color='#0D0D0C', alpha=1, lw=2, label='(AUC = {:.2f} [{:.2f}])'.format(np.mean(metric_auc_roc_list), np.std(metric_auc_roc_list)))
    plt.legend(loc="lower right")

    title_metrics = '\n' + 'acc'
    plt.title('ROC ('+title+')\n acc={:.2f}  prec={:.2f}  f1={:.2f} rec={:.2f}'.format(np.mean(acc_list), 
                                                                                np.mean(prec_list),
                                                                                np.mean(f1_list), np.mean(rec_list)))

In [ ]:
y_ = df['chronic_pain']
X_ = df.drop(['chronic_pain'], axis=1)


columns_to_normalize = ['sex', 'age', 'reduc_mob', 'depression', 'anxiety', 'malnutrition', 'educ_yrears', 'SES']
scaler = MinMaxScaler([0.05, 0.95])

X_s = X_.copy()
X_s[columns_to_normalize]  = scaler.fit_transform(X_[columns_to_normalize])
#X_s = pd.DataFrame(scaling_data, columns= X_.columns, index = X_.index)

X_ =  X_s.copy()

opt_XGB = SearchBestModel(X_.drop(['factor'], axis = 1), y_)
params_ = dict(opt_XGB.best_params_)
X_;

In [ ]:
X_

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

undersample = RandomUnderSampler(sampling_strategy='auto', random_state=10)
X_resampled, y_resampled = undersample.fit_resample(X_, y_)


In [ ]:
for i in range(100):
    try:
        classifier_ROC(X_resampled, y_resampled, False, None, i);
    except:
        pass

In [ ]:
r = classifier_ROC(X_resampled, y_resampled, False, None, 20)

In [ ]:
pain_results = {}
pain_results['chronic_pain'] = r

In [ ]:
plt.figure(figsize=(4,4))
graph_roc(pain_results, ['chronic_pain'], title = 'NHS 2016')
plt.savefig('Paper_figures/pdfs/ROC_2016.pdf', format='pdf')

In [ ]:
plt.figure(figsize=(3,8))
graph_feat_import(pain_results, ['chronic_pain'], colores=['gray'])
plt.savefig('Paper_figures/pdfs/feat_2016.pdf', format='pdf')